<a href="https://colab.research.google.com/github/leoagneau/Bib_Racer/blob/master/Photos_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import urllib
import time
import os
import requests

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Set data path
root_dir = '/content/drive/My Drive/'
data_path = root_dir + 'ML_Projects/Bib_Racer/data/'
img_path = data_path + 'images/'
if not os.path.exists(img_path):
  os.makedirs(img_path)
os.chdir(img_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Image scrapping
Images avaialble at:
- https://hk.running.biji.co/index.php?q=album&act=gallery_album&competition_id=1734
- https://www.sportsoho.com/pg/photos/matchalbum/7334600
- https://www.run-pic.com/EventAlbum.aspx?eventId=734


### Install and load Selenium

In [0]:
# https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
# install chromium, its driver, and selenium:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

#import sys
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# set options to be headless
from selenium import webdriver
sln_options = webdriver.ChromeOptions()
sln_options.add_argument('--headless')
sln_options.add_argument('--no-sandbox')
sln_options.add_argument('--disable-dev-shm-usage')

### Scrapping site 1 - running biji

In [0]:
def soup_page(page):
    try:
      return BeautifulSoup(page, 'lxml')
    except:
      print("Cannot fetch the requested page")

In [0]:
# Open album page
url_bj = "https://hk.running.biji.co/index.php?q=album&act=gallery_album&competition_id=1734"
albums_page = urllib.request.urlopen(url_bj)
soup = soup_page(albums_page)

In [0]:
# Locate albums section and retrieve all album links
albums_list = soup.find('ul', attrs={'class': 'gal-grid'})
lnks = albums_list.find_all('a')
album_pages = []
for l in lnks:
  album_pages.append('https://hk.running.biji.co'+l['href'])
print('{} albums found.'.format(len(album_pages)))

75 albums found.


In [0]:
# Scrape dynamic length page by simulating mouse scrolling https://stackoverflow.com/questions/49088880/scraping-all-results-from-page-with-beautifulsoup
def load_page(url):
    driver = webdriver.Chrome('chromedriver', options=sln_options)
    driver.get(url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    #print(last_height)
    #i=0
    while True:
      #i+=1
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(1.0)
      new_height = driver.execute_script("return document.body.scrollHeight")
      #print("{}-th scroll, new height={}".format(i, new_height))
      if new_height == last_height:
        break
      last_height = new_height
    #print(last_height)
    return driver

In [0]:
a_num=0
for album in album_pages:
  photo_page = load_page(album)
  a_num+=1
  # Retrieve content of the album
  soup = soup_page(photo_page.page_source)
  photo_list = soup.find_all('img', attrs={'class': 'photo_img'})
  print('Album {} contains {} photos.'.format(a_num, len(photo_list)))
  # Download photos from an album
  for i in range(len(photo_list)):
    lnk = photo_list[i]['src'].replace("600", "1024")
    with open(img_path + os.path.basename(lnk), "wb") as f:
      f.write(requests.get(lnk).content)
  print('Finished processing album {}'.format(a_num))

Album 1 contains 355 photos.
Finished processing album 1
Album 2 contains 406 photos.
Finished processing album 2
Album 3 contains 573 photos.
Finished processing album 3
Album 4 contains 410 photos.
Finished processing album 4
Album 5 contains 450 photos.
Finished processing album 5
Album 6 contains 492 photos.
Finished processing album 6
Album 7 contains 498 photos.
Finished processing album 7
Album 8 contains 200 photos.
Finished processing album 8
Album 9 contains 496 photos.
Finished processing album 9
Album 10 contains 260 photos.
Finished processing album 10
Album 11 contains 321 photos.
Finished processing album 11
Album 12 contains 100 photos.
Finished processing album 12
Album 13 contains 403 photos.
Finished processing album 13
Album 14 contains 413 photos.
Finished processing album 14
Album 15 contains 412 photos.
Finished processing album 15
Album 16 contains 276 photos.
Finished processing album 16
Album 17 contains 436 photos.
Finished processing album 17
Album 18 contai